## Bootstrapping sEEG electrode significance
We can use a bootstrap _t_-test if we determine a proper baseline in the continuous data. This technique can be seen in [Cheung et al. 2016 _eLife_](https://elifesciences.org/articles/12577):

> To identify if a site was responsive to speech sounds, we implemented a bootstrap t-test comparing a site's responses randomly sampled over time during speech sound presentations to responses randomly sampled over time during pre-stimulus silent intervals (p<0.01).

In the context of this task we would like to calculate a separate p value for the baseline of the perception trials and the baseline of the production trials. For both of these, the period after the click (but not immediately following) is the best candidate. We can get the baseline times by epoching the clicks 500-1000ms. We choose 1000ms as a cutoff because that is when display of the fixation cross ends and either text is displayed (production) or audio playback begins (perception). Because we have a variable amount of clicks per subject we are going to bootstrap with replacement for all subjects.

In [ ]:
# Paths - Update locally!
git_path = '/path/to/git/kurteff2024_code/'
data_path = '/path/to/bids/dataset/'

In [ ]:
import mne
import numpy as np
import pandas as pd
import os
import re
import csv
from tqdm.notebook import tqdm
import warnings

from img_pipe import img_pipe

from matplotlib import pyplot as plt
from matplotlib import rcParams as rc
import matplotlib.patheffects as PathEffects
rc['pdf.fonttype'] = 42
plt.style.use('seaborn')
%matplotlib inline

In [ ]:
subjs = [s for s in os.listdir(
    os.path.join(git_path,"preprocessing","events","csv")) if "TCH" in s or "S0" in s]
exclude = ["TCH8"]
no_imaging = ["S0010"]
subjs = [s for s in subjs if s not in exclude]

blocks = {
    s: [
        b.split("_")[-1] for b in os.listdir(os.path.join(
            git_path,"analysis","events","csv",s)) if f"{s}_B" in b and os.path.isfile(os.path.join(
            git_path,"analysis","events","csv",s,b,f"{b}_spkr_sn_all.txt"
        ))
    ] for s in subjs
}

### Epoch data

In [ ]:
click_tmin, click_tmax = 0.4, 0.6
task_tmin, task_tmax = 0.05, 0.55
baseline = None
epochs, ch_names = dict(), dict()
for s in tqdm(subjs):
    epochs[s] = dict()
    click_epochs, spkr_epochs, mic_epochs = [], [], []
    for b in blocks[s]:
        blockid = "_".join([s,b])
        raw = mne.io.read_raw_fif(os.path.join(data_path,f"{s}_complete",s,blockid,
            "HilbAA_70to150_8band","ecog_hilbAA70to150.fif"),preload=True, verbose=False)
        fs = raw.info['sfreq']
        # Click events
        click_eventfile = os.path.join(git_path,"preprocessing","events","csv",s,blockid,
                                       f"{blockid}_click_eve.txt")
        with open(click_eventfile,'r') as f:
            r = csv.reader(f, delimiter='\t')
            if s not in ["S0026","TCH14"]:
                events = np.array([[np.ceil(float(row[0])*fs).astype(int),
                                    np.ceil(float(row[2])*fs).astype(int),int(float(row[4]))] for row in r])
            else:
                events = np.loadtxt(click_eventfile).astype(int)
                events[:,0] = np.array(events[:,0] * fs).astype(int)
                events[:,1] = np.array(events[:,1] * fs).astype(int)
        click_epochs.append(mne.Epochs(raw, events, tmin=click_tmin, tmax=click_tmax, baseline=baseline,
            preload=True, verbose=False))
        # Spkr events
        spkr_eventfile = os.path.join(git_path,"preprocessing","events","csv",s,blockid,
                                      f"{blockid}_spkr_sn_all.txt")
        with open(spkr_eventfile,'r') as f:
            r = csv.reader(f, delimiter='\t')
            events = np.array([[np.ceil(float(row[0])*fs).astype(int),
                                np.ceil(float(row[1])*fs).astype(int),int(row[2])] for row in r])
        spkr_epochs.append(mne.Epochs(raw, events, tmin=task_tmin, tmax=task_tmax, baseline=baseline,
                                      preload=True, verbose=False))
        # Mic events
        mic_eventfile = os.path.join(git_path,"preprocessing","events","csv",s,blockid,
                                     f"{blockid}_mic_sn_all.txt")
        with open(mic_eventfile,'r') as f:
            r = csv.reader(f, delimiter='\t')
            events = np.array([[np.ceil(float(row[0])*fs).astype(int),
                                np.ceil(float(row[1])*fs).astype(int),int(row[2])] for row in r])
        mic_epochs.append(mne.Epochs(raw, events, tmin=task_tmin, tmax=task_tmax, baseline=baseline,
                                     preload=True, verbose=False))
    epochs[s]['click'] = mne.concatenate_epochs(click_epochs)
    epochs[s]['spkr'] = mne.concatenate_epochs(spkr_epochs)
    epochs[s]['mic'] = mne.concatenate_epochs(mic_epochs)
    ch_names[s] = raw.info['ch_names']

### Run the bootstrap

In [ ]:
nboots = 1000; pvals = dict()
pvals['spkr'] = {s:{ch:0 for ch in ch_names[s]} for s in subjs}
pvals['mic'] = {s:{ch:0 for ch in ch_names[s]} for s in subjs}
for s in tqdm(subjs):
    baseline = epochs[s]['click'].get_data()
    spkr_resp = epochs[s]['spkr'].get_data(); mic_resp = epochs[s]['mic'].get_data()
    nepochs_click = baseline.shape[0]; nepochs_spkr = spkr_resp.shape[0]; nepochs_mic = mic_resp.shape[0]
    for ch_idx, ch in enumerate(ch_names[s]):
        spkr_ch_boots, mic_ch_boots = [], []
        for n in np.arange(nboots):
            # Get a random ten epochs and average across these
            baseline_idx = np.random.choice(np.arange(nepochs_click), size=10)
            mic_idx = np.random.choice(np.arange(nepochs_mic), size=10)
            spkr_idx = np.random.choice(np.arange(nepochs_spkr), size=10)
            # Average resp across samples/epochs (chunk of 10)
            baseline_chunk = baseline[baseline_idx,ch_idx,:].mean(0).mean(0)
            spkr_chunk = spkr_resp[spkr_idx,ch_idx,:].mean(0).mean(0)
            mic_chunk = mic_resp[mic_idx,ch_idx,:].mean(0).mean(0)
            if baseline_chunk > spkr_chunk:
                spkr_ch_boots.append(1)
            else:
                spkr_ch_boots.append(0)
            if baseline_chunk > mic_chunk:
                mic_ch_boots.append(1)
            else:
                mic_ch_boots.append(0)
        pvals['spkr'][s][ch] = sum(spkr_ch_boots)/nboots; pvals['mic'][s][ch] = sum(mic_ch_boots)/nboots

In [ ]:
# Save to dataframe
df = pd.DataFrame(columns = ['subj', 'ch_name', 'spkr_p', 'mic_p'])
for s in pvals['spkr'].keys():
    ch_names = list(pvals['spkr'][s])
    for ch in ch_names:
        new_row = pd.DataFrame({'subj':[s], 'ch_name':[ch],
                                'spkr_p':[pvals['spkr'][s][ch]], 'mic_p':[pvals['mic'][s][ch]]})
        df = df.append(new_row, ignore_index=True)
df.to_csv(os.path.join(git_path,"stats","bootstraps","csv",
    f"seeg_elec_significance_{len(pvals['spkr'].keys())}_subjs_{nboots}_boots.csv"), index=False)  